In [ ]:
from pathlib import Path

# Paste output from benchmark binary. This notebook parses and plots the benchmark results.
benchmark_output = Path(f"benchmark_output/output.json")
if not benchmark_output.is_file():
    benchmark_output = Path(f"benchmark_output_example/output.json")

save_images_used_in_readme = True
figure_show_mode = 'svg' # 'svg' is inline and visible in GitHub, None is native interactive plotly plots that are not visible in GitHub and several MB in size

In [ ]:
def google_benchmark_output_to_dataframe(path: Path):
    import json

    everything = json.loads(path.read_text())

    sequential = []
    max_p = 1
    # parse out interesting information
    for bench in everything["benchmarks"]:
        # is the benchmark parallel
        bench["parallel"] = "p:" in bench["name"]

        name_parts = bench["name"].split("/")

        bench["p"] = 1
        for part in name_parts:
            if ":" in part:
                k, v = part.split(':')
                bench[k] = int(v)
        max_p = max(max_p, bench["p"])

        name = name_parts[0]
        bench["name"] = name

        if " " in name:
            bench["op"] = name.split(" ")[0]
            bench["variant"] = name.split(" ")[1]
        else:
            bench["op"] = name

        if not bench["parallel"]:
            sequential.append(bench)

    for bench in sequential:
        dupe = dict(bench)
        dupe["p"] = max_p
        everything["benchmarks"].append(dupe)

    res = pd.DataFrame(everything["benchmarks"])
    return res


In [ ]:
import pandas as pd
import plotly.express as px

df = google_benchmark_output_to_dataframe(benchmark_output)
df.sort_values(by=['name', 'p'], ascending=True, inplace=True)

In [ ]:

for variant in "read", "write":
    data = df.copy()
    data = data[data["variant"] == variant]
    data["op_disp"] = data["op"].apply(lambda op: "IOStream Triplet" if op == "IOStream" else f"FMM {op}")
    data.sort_values(by=['op_disp', 'p'], ascending=True, inplace=True)

    fig = px.line(data, x='p', y='bytes_per_second', title=f"{variant.title()} Parallelism", color="op_disp")
    fig.update_layout(
        yaxis = dict(
            title=None,
            ticksuffix = 'B/s',
            rangemode = 'tozero',
            tickformat = '~s'
        ),
        xaxis = dict(title='# of threads'),
        legend=dict(
            title=None,
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
    )
    fig.show(figure_show_mode)
    if save_images_used_in_readme:
        fig.write_image(f"parallel-scaling-{variant}.svg")